## Задание 1

Задание про светку списка 

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 10, [1, 2, 3])

16

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '=', [1, 2, 3])

'=123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

foldl2

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: f(y, x), x0, lst[::-1])

In [6]:
#сумма
foldl2(lambda x, y: x + y, 10, [1, 2, 3])

16

In [7]:
#конкатенация
foldl2(lambda x, y: '{}{}'.format(x, y), '=', [1, 2, 3])

'=123'

In [8]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

foldr2

In [9]:
def foldr2(f, x0, lst):
    return foldl(lambda x, y: f(y, x), x0, lst[::-1])

In [10]:
foldr(lambda x, y: '{}{}'.format(x, y), '=', [1, 2, 3])

'123='

In [11]:
#конкатенация
foldr2(lambda x, y: '{}{}'.format(x, y), '=', [1, 2, 3])

'123='

In [12]:
f = lambda x, y: x / y
foldr2(f, 1, [1, 2, 3])

1.5

## Задание 2

Проверка на то входит ли b как перестановка в a

Оценка сложности. В python словари реализованы через HashMap, а значит поиск по ключу, добавление ключа, изменение значения по ключу работают за $O(1)$. Проверка двух словарей на равенство тогда работает за время пропорциональное алфавиту, в данном случае я предполагаю алфавит a-zA-Z, тогда сравнение тоже работает за константное время. Тогда у нас остается один цикл код внутри которого имеет сложность $O(1)$, а значит мой код работает за $O(len(a))$ (Я предполагаю что len a >= len b)

In [13]:
import string

def check_inv(a, b):
    assert len(a) >= len(b)
    
    b_chars = dict.fromkeys(string.ascii_letters, 0)
    for char in b:
        b_chars[char] += 1
            
    curr_chars = dict.fromkeys(string.ascii_letters, 0)
    for char in a[:len(b) - 1]:
        curr_chars[char] += 1
            
    for i in range(len(b) - 1, len(a)):
        curr_chars[a[i]] += 1
        if(curr_chars == b_chars):
            return True
        curr_chars[a[i - len(b) + 1]] -= 1
        
    return False

In [14]:
a = 'rtabcromaaator'
b = 'tro'

check_inv(a, b)

True

In [15]:
a = 'rotabcromaaatr'
b = 'tro'

check_inv(a, b)

True

In [16]:
a = 'rtabcrotmaaatr'
b = 'tro'

check_inv(a, b)

True

## Задание 3

Бинарное дерево 

In [17]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if(self.left == None and self.right == None):
            yield self.value
            
        if(self.left):
            yield from self.left
            
        if(self.right):
            yield from self.right
        
    
    def __str__(self):
        res = [str(self.value)]
        
        if(self.left):
            res.append(self.left.__str__())
            
        if(self.right):
            res.append(self.right.__str__())
            
        return ' '.join(res)
    
    def __repr__(self):
        if(self.left == None and self.right == None):
            return f"Tree({self.value})"
        
        if(self.right == None):
            return f"Tree({self.value}, {self.left.__repr__()})"
        
        if(self.left == None):
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
 
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [18]:
# __str__ test
print(tree)

0 1 3 4 2


In [19]:
# __repr__ test
tree

Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

In [20]:
# __repr__ eval test
tree_repr_test = eval(tree.__repr__())
print(tree_repr_test)

0 1 3 4 2


In [21]:
# __iter__ test
for it in tree:
    print(it)

3
4
2


## Задание 4 

Калькулятор 

In [22]:
def parse(expr):
    buf = []

    for val in expr.split(" "):
        if(val[0] == "("):
            buf.append("(")
            buf.append(val[1:])
        elif(val[-1] == ")"):
            buf.append(val[:-1])
            buf.append(")")
        else:
            buf.append(val)

    result = []
    stack = []
    high_priority = set(["*", "/"])
    low_priority = set(["+", "-"])
    
    for val in buf:
        if(val.isnumeric()):
            result.append(val)
        if(val == "("):
            stack.append(val)
        if(val == ")"):
            while(stack[-1] != "("):
                result.append(stack.pop())
            stack.pop()
        if(val in low_priority):
            while(len(stack) > 0 and stack[-1] in high_priority):
                result.append(stack.pop())
            stack.append(val)
        if(val in high_priority):
            stack.append(val)
            
    while(len(stack) > 0):
        result.append(stack.pop())
    
    return result

def calc(expr):
    operations = {
        "+": lambda x, y: x + y,
        "-": lambda x, y: x - y,
        "*": lambda x, y: x * y,
        "/": lambda x, y: x / y
    }
    
    expr = parse(expr)
    stack = []
    
    for val in expr:
        if(val.isnumeric()):
            stack.append(int(val))
        if(val in operations):
            right = stack.pop()
            left = stack.pop()
            stack.append(operations[val](left, right))
    return stack.pop()

In [23]:
calc('2 * (15 - 3 * 4) - 2') == 4

True

In [24]:
calc('3 + 4 * 2 / (1 - 5)')

1.0